# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher_20.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name='Reacher_20.app')

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
import os.path

def restore_agent(actor_name, filepath_local_actor, filepath_local_critic, filepath_target_actor, filepath_target_critic):
    # function to read and load saved weights into agent networks
    
    checkpoint_local_actor = torch.load(filepath_local_actor)
    checkpoint_local_critic = torch.load(filepath_local_critic)
    checkpoint_target_actor = torch.load(filepath_target_actor)
    checkpoint_target_critic = torch.load(filepath_target_critic)
    
    if actor_name == 'ddpg':
        loaded_agent = Agent(state_size, action_size, random_seed = 33)
    elif actor_name == 'td3':
        loaded_agent = Agent(state_size, action_size, random_seed = 33, policy_noise=0.2)
    elif actor_name == 'd4pg':
        loaded_agent = Agent(state_size, action_size, random_seed = 33)
    
    loaded_agent.actor_local.load_state_dict(checkpoint_local_actor)
    loaded_agent.actor_target.load_state_dict(checkpoint_target_actor)
    loaded_agent.critic_local.load_state_dict(checkpoint_local_critic)
    loaded_agent.critic_target.load_state_dict(checkpoint_target_critic)
    
    return loaded_agent

In [ ]:
from collections import deque
import torch

def run_experiment(agent, n_episodes=2000, max_t=10000, agent_ckp_prefix='agent', critic_ckp_prefix='critic'):
    
    scores_deque = deque(maxlen=100)
    rolling_average_score = []
    scores = []
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                 # get the current state
        score = np.zeros(num_agents)
        agent.reset()                                         # reset the agent
        for t in range(max_t):
            actions = agent.act(states, add_noise=False)        
             
            env_info = env.step(actions)[brain_name]              # send all actions to tne environment
            next_states = env_info.vector_observations            # get next state (for each agent)
            rewards = env_info.rewards                            # get reward (for each agent)
            dones = env_info.local_done                           # see if episode finished                
  
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done, t)

            states = next_states
            score += rewards
            
            if np.any(dones):
                break
        
        score = np.mean(score)
        scores_deque.append(score)
        rolling_average_score.append(np.mean(scores_deque))
        scores.append(score)
          
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, 
                                                                          np.mean(scores_deque), 
                                                                          score), end='')
        
        if i_episode % 10 == 0:   
            print('\rSave_agent\r')
            torch.save(agent.actor_local.state_dict(), agent_ckp_prefix+'_ckpt_local.pth')                 # save local actor
            torch.save(agent.actor_target.state_dict(), agent_ckp_prefix+'_ckpt_target.pth')               # save target actor 
            torch.save(agent.critic_local.state_dict(), critic_ckp_prefix+'_ckpt_local.pth')               # save local critic
            torch.save(agent.critic_target.state_dict(), critic_ckp_prefix+'_ckpt_target.pth')             # target critic
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque))) 
        if np.mean(scores_deque)>= 30.0 and i_episode > 100:
            torch.save(agent.actor_local.state_dict(), agent_ckp_prefix+'_ckpt_local.pth')                 # save local actor
            torch.save(agent.actor_target.state_dict(), agent_ckp_prefix+'_ckpt_target.pth')               # save target actor 
            torch.save(agent.critic_local.state_dict(), critic_ckp_prefix+'_ckpt_local.pth')               # save local critic
            torch.save(agent.critic_target.state_dict(), critic_ckp_prefix+'_ckpt_target.pth')       
            print('\rEnvironment solved Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque))) 
            break
    return scores, rolling_average_score

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

## DDPG

The first algorithm will be a standard DDPG as found in the examples from the 
Udacity DeepLearning NanoDegree: https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-bipedal

It will be adapted to solve the Reacher environment

In [ ]:
from actors.ddpg_actor import Agent

agent_name = 'checkpoints/agent_ddpg_20'
critic_name = 'checkpoints/critic_ddpg_20'

local_actor_path = agent_name+'_ckpt_local.pth'
target_actor_path = agent_name+'_ckpt_target.pth'
local_critic_path = critic_name+'_ckpt_local.pth'
target_critic_path = critic_name+'_ckpt_target.pth'

        
# if checkpoint exists we load the agent 
if os.path.isfile(local_actor_path):
    agent = restore_agent('ddpg', local_actor_path, local_critic_path, target_actor_path, target_critic_path)
    print("Agent loaded.") 
else:
    agent = Agent(state_size, action_size, random_seed = 33)
    print("Agent created.")

In [ ]:
scores, rolling_average= run_experiment(agent, agent_ckp_prefix=agent_name, critic_ckp_prefix=critic_name)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot scores across episodes
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='TD3')
plt.plot(np.arange(len(scores)), rolling_average, c='r', label='Rolling AVG')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left');
plt.show()

In [ ]:
env.close()

## TD3 
The first improvement tried is the TD3 algorithm which essentially make 3 improvements to the DDPG.
1. Twin network for the critic
2. Add noise to actions used to compute targets
3. Delayed updates of the policy

Please restart the invirnonment before running the cells below

In [ ]:
from actors.td3_actor import Agent

agent_name = 'checkpoints/agent_td3_20'
critic_name = 'checkpoints/critic_td3_20'

local_actor_path = agent_name+'_ckpt_local.pth'
target_actor_path = agent_name+'_ckpt_target.pth'
local_critic_path = critic_name+'_ckpt_local.pth'
target_critic_path = critic_name+'_ckpt_target.pth'

        
# if checkpoint exists we load the agent
if os.path.isfile(local_actor_path):
    agent = restore_agent('td3', local_actor_path, local_critic_path, target_actor_path, target_critic_path)
    print("Agent loaded.") 
else:
    agent = Agent(state_size, action_size, random_seed = 33)
    print("Agent created.")

In [ ]:
# run
scores, rolling_average = run_experiment(agent, agent_ckp_prefix=agent_name, critic_ckp_prefix=critic_name)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot scores across episodes
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='TD3')
plt.plot(np.arange(len(scores)), rolling_average, c='r', label='Rolling AVG')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left');
plt.show()

In [ ]:
env.close()

## D4PG
State of the art improvements over DDPG


Please restart the invirnonment before running the cells below

In [ ]:
from actors.d4pg_actor import Agent
import os.path

agent_name = 'checkpoints/agent_d4pg_20'
critic_name = 'checkpoints/critic_d4pg_20'

local_actor_path = agent_name+'_ckpt_local.pth'
target_actor_path = agent_name+'_ckpt_target.pth'
local_critic_path = critic_name+'_ckpt_local.pth'
target_critic_path = critic_name+'_ckpt_target.pth'

        
# if checkpoint exists we load the agent
if os.path.isfile(local_actor_path):
    agent = restore_agent('d4pg', local_actor_path, local_critic_path, target_actor_path, target_critic_path)
    print("Agent loaded.") 
else:
    agent = Agent(state_size, action_size, random_seed = 33)
    print("Agent created.")

In [ ]:
# run
scores, rolling_average = run_experiment(agent, agent_ckp_prefix=agent_name, critic_ckp_prefix=critic_name)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot scores across episodes
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='D4PG')
plt.plot(np.arange(len(scores)), rolling_average, c='r', label='Rolling AVG')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left');
plt.show()

In [ ]:
env.close()